# Object Detection Federated CNN Server Side
This code is the server part of Car Detection federated CNN model for **multi** client and a server.

## Setting variables

In [1]:
import os
import sys
import time
import copy
import h5py
import struct
import socket
import pickle
from tqdm import tqdm

import torch
import torch.nn as nn
from threading import Lock
import torch.optim as optim
from threading import Thread
import torch.nn.functional as F

In [2]:
import cv2
import ast
import math
import matplotlib
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import os,sys,matplotlib,re
from skimage import exposure
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as immg
import torchvision.transforms as T
from torchvision.io import read_image
from collections import defaultdict, deque
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import draw_bounding_boxes
from torchvision.models.detection import FasterRCNN
from torch.utils.data.sampler import SequentialSampler
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import warnings
warnings.filterwarnings("ignore")

## Declaring Device

In [3]:
# Checking if cuda is available 
#The code will use the GPU if available, otherwise it will use the CPU.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [4]:
users = 2 # number of clients
rounds = 3 # Number of epochs
local_epoch = 1

## CNN Model for object Detection

In [5]:
# Defining Pytorch CNN pretrained model
#The code attempts to create a model that will predict the class score of an image based on the input image.
def model1(num): #it creates a model with the name of "model1" and sets its input to be num.
    #it assigns this object to variable named model1 so that we can use it later on in our code.
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True) #It then defines a function called "model".
    #is an instance of torchvision's class called "DetectionModel", which contains information about how many features are used for prediction (in this case, 50).
    in_features = model.roi_heads.box_predictor.cls_score.in_features #we create another variable called in_features and assign it to be equal to what was just assigned to variable model1 - i.e., 50 features were used for prediction here because there are 50 boxes in each image!
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num) #This function has two parameters: in_features and num.
    return model

In [6]:
# Creating object of model
resnet_cnn = model1(2)

In [7]:
# Sending device to device(cpu or gpu)
#The code will send the current device's IP address to CNN.
resnet_cnn.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

## Variables

In [8]:
# Declaring required Variables
# The input list contains three variables: start_time, weightcount, and lock; while the output list contains one variable: datasetsize.
#The code attempts to create a copy of the ResNet CNN object, and then use it to initialize the global_weights variable.
start_time = 0 #This is used to keep track of the time that has passed since the program started running.
weight_count = 0 #It will hold all the users in this dataset.
lock = Lock()
datasetsize = [0]*users
weights_list = [0]*users #It will hold all the weight values for each user.
clientsoclist = [0]*users #it creates an array called clientsoclist which will hold all these weight values for each client (or node).

# Copying original weights of our model
global_weights = copy.deepcopy(resnet_cnn.state_dict())

## Overhead of Communication

In [9]:
# Defining lists for further use
total_sendsize_list = [] #creating a list of users (users) and their total sendsize 
total_receivesize_list = [] #creating a list of users (users) and their total receivesize 
train_sendsize_list = [] #It creates list  with all of the send sizes that each user has sent
train_receivesize_list = [] #It creates list  with all of the Receive sizes that each user has sent
client_receivesize_list = [[] for i in range(users)] #it loops through these lists to create a dictionary where keys are users and values are lists containing send size or receive size
client_sendsize_list = [[] for i in range(users)] #it loops through these lists to create a dictionary where keys are users and values are lists containing send size or receive size

### Socket functions

In [10]:
# Helper function to receive n bytes or return None if EOF is hit
def RECVALL(sock, n): #defining a function called RECVALL.
    data = b'' #The code declares the variable data to be of type b'' which is short for "byte array".
    while len(data) < n: #creates a while loop that will continue until it has received less than n bytes from the socket.
        packet = sock.recv(n - len(data)) #reads all the bytes from the socket into packet using recv(n - len(data)).
        if not packet: 
            return None
        data += packet #After reading all the bytes into packet, data is set equal to packet + data and continues on with its looping process until it reaches less than n bytes again or when it receives None back from recvall().
    return data

# Read message length and unpack it into an integer
#defines a function that takes in a sock as one parameter and returns either None or some string depending on whether there was any error during sending or receiving packets over this particular connection.
def MSG_RECV(sock):
    raw_msglen = RECVALL(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # Read the message data
    msg =  RECVALL(sock, msglen)
    msg = pickle.loads(msg)
    return msg, msglen

# Prefix each message with a 4-byte length in network byte order
#another function that sends out strings through sockets based on how many characters are being sent at once (l_send) plus whatever message they want to send (msg).
def MSG_SEND(sock, msg):
    msg = pickle.dumps(msg)
    l_send = len(msg)
    msg = struct.pack('>I', l_send) + msg
    sock.sendall(msg)
    return l_send
#The code attempts to send a message of length 4 bytes.

In [11]:
# Returns the average of the weights
#The code iterates through the list of weights, and for each weight it multiplies the value by the corresponding data.
#The result is then divided by the sum of all values to get an average.
#The code iterates through a list of weights and for each weight it multiplies its value by that weight's corresponding data element.
#Then, it divides this product by the sum of all values in order to get an average.
def Average_of_Weights(w, datasize):
        
    for i, data in enumerate(datasize):
        for key in w[i].keys():
            w[i][key] *= float(data)
    
    w_avg = copy.deepcopy(w[0])

    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], float(sum(datasize)))
        
    return w_avg

## Define Thread

## Receiving Users just before starting Training

In [12]:
# Function to Keep server running 
def run_thread(func, num_user): ##creating a list of global variables.
    global clientsoclist  #will be used to keep track of all the connections that are made during this program.
    global start_time #to store when the program started and end_time which stores when it ends.
    
    thrs = [] #we create an empty list called thrs and then iterate through num_user times adding each connection to clientsoclist
    for i in range(num_user):
        conn, addr = s.accept()
        print('Conntected with', addr)
        # append client socket on list
        clientsoclist[i] = conn
        args = (i, num_user, conn) ##it creates a function that takes three parameters and returns the result of calling that function on each user's connection.
        thread = Thread(target=func, args=args) #we create a Thread object where func is the function that needs to run on every thread in thrs.
        thrs.append(thread) # we append our new thread into thrs so that it can run on its own after starting it up.
        thread.start() #The code attempts to train a threading application with the given number of users.
    #The code starts by creating an empty list for the threads, which will be filled in later.
    print("timmer start!")
    start_time = time.time()    # store start time
    for thread in thrs:
        thread.join()
    end_time = time.time()  # store end time
    print("TrainingTime: {} sec".format(end_time - start_time))
    #It then loops through each thread in turn and calls its join() method once finished, which causes all threads to exit gracefully.

In [13]:
# Function to receive clients
def receive(userid, num_users, conn): #The code starts by creating a dictionary with the keys being userid and rounds.
    global weight_count
    global datasetsize 
    msg = {
        'rounds': rounds,
        'client_id': userid,
        'local_epoch': local_epoch
    }

    # send epoch
    datasize = MSG_SEND(conn, msg) 
    total_sendsize_list.append(datasize) #It will contain all of the sizes for every message sent to every client.
    client_sendsize_list[userid].append(datasize) #It will contain all of the sizes for every message sent from each individual client.

    # get total_batch of train dataset
    train_dataset_size, datasize = MSG_RECV(conn) #locks on datasetsize so that no other process can change its value while this function is running.
    #The values are lists of integers that represent how many times each user has been sent messages in this round.  
    total_receivesize_list.append(datasize) #It represents how much data has been received from our training dataset
    client_receivesize_list[userid].append(datasize) #how much data has been received from our clients (the number of users).
    
    with lock:
        datasetsize[userid] = train_dataset_size
        weight_count += 1 #we add 1 to weight count because there is one extra element in both lists now after adding 1 to datasetsize[userid].
    
    train(userid, train_dataset_size, num_users, conn) # called train() with these new values for num_users and conn so that it can start
    #The code attempts to send a message to the server with the following information: rounds: rounds, client_id: userid, local_epoch: local_epoch.

## Train

In [14]:
# Function for Training
# The train_dataset_size variable is defined as being equal to MSG_SEND(conn, global_weights) which will send out one dataset size worth of data from each client.
#The total number of users in this experiment is defined as num_users, so there will be a total number of rounds equal to rounds.

def train(userid, train_dataset_size, num_users, client_conn): #defining the variables that are used in the program.
    global weights_list
    global global_weights #It  is a list of all weights for each user, and it's initialized to an empty list at the beginning of every round.
    global weight_count
    global resnet_cnn #The resnetcnn classifier was created using Keras and trained on ImageNet with batch normalization and dropout layers.
    #It has been pre-trained on ImageNet with over 1 billion images labeled into 1000 categories (1000 classes).
    global val_acc
    
    for r in range(rounds):
        with lock: #called lock which will prevent multiple threads from accessing it simultaneously while they're iterating through their respective loops.
            if weight_count == num_users: #two nested loops: one for sending messages between clients and another for receiving messages from clients.
                for i, conn in enumerate(clientsoclist):
                    datasize = MSG_SEND(conn, global_weights)
                    total_sendsize_list.append(datasize)
                    client_sendsize_list[i].append(datasize)
                    train_sendsize_list.append(datasize)
                    weight_count = 0
    #The code is a snippet of code that is used to train the weights for the neural network.
        client_weights, datasize = MSG_RECV(client_conn)
        total_receivesize_list.append(datasize)
        client_receivesize_list[userid].append(datasize)
        train_receivesize_list.append(datasize)

        weights_list[userid] = client_weights
        print("User" + str(userid) + "'s Round " + str(r + 1) +  " is done")
        with lock:
            weight_count += 1
            if weight_count == num_users:
                #average
                global_weights = Average_of_Weights(weights_list, datasetsize)
    #The code starts by checking if there are enough users in the dataset.
    # If there are not enough users, then it will loop until all users have been processed.
    #it will create a list of clients and send them data from the global_weights variable.
    #Then, it will check if there are enough clients and repeat this process until all clients have been sent data from the global_weights variable.


## Socket initialization
### Set host address and port number

In [15]:
# The code prints the hostname of the machine on which Python is running.
host = socket.gethostbyname(socket.gethostname())
port = 10087
print(host)

192.168.43.128


In [16]:
s = socket.socket() #creating a socket object.
s.bind((host, port)) #binds it to the specified host and port.
s.listen(5) 
#This causes the program to wait for incoming connections from other computers (or clients) that are connected to this computer's network interface card (NIC).

### Open the server socket

In [17]:
run_thread(receive, users)
#calling receive(), which will be executed in a separate thread of execution.
#calling users(), which will also be executed in its own thread of execution.
#The code attempts to create a server that listens on port 10087 or whatever port we give.

Conntected with ('192.168.43.35', 59879)
Conntected with ('192.168.43.128', 62174)
timmer start!


In [ ]:
# The code computes the time taken for a training session.
end_time = time.time()  # store end time
print("TrainingTime: {} sec".format(end_time - start_time))

TrainingTime: 464.98293137550354 sec


### Saving the model

In [ ]:
PATH = './FL_fasterrcnn_car_detect.pth'
torch.save(resnet_cnn.state_dict(), PATH)

## Print all of communication overhead

In [ ]:
#The code starts by printing out the total_sendsize_list and the total_receivesize_list.
#The code then prints out a list of all the users in order, followed by their client sendsizes and receivesizes.
#The code starts with a for loop that iterates through each user in order to print out their client sendsizes and receivesizes.
#The code will print total size of send and received data for each user.

print('\n')
print('---total_sendsize_list---')
total_size = 0
for size in total_sendsize_list:

    total_size += size
print("total_sendsize size: {} bytes".format(total_size))
print('\n')

print('---total_receivesize_list---')
total_size = 0
for size in total_receivesize_list:

    total_size += size
print("total receive sizes: {} bytes".format(total_size) )
print('\n')

for i in range(users):
    print('---client_sendsize_list(user{})---'.format(i))
    total_size = 0
    for size in client_sendsize_list[i]:

        total_size += size
    print("total client_sendsizes(user{}): {} bytes".format(i, total_size))
    print('\n')

    print('---client_receivesize_list(user{})---'.format(i))
    total_size = 0
    for size in client_receivesize_list[i]:

        total_size += size
    print("total client_receive sizes(user{}): {} bytes".format(i, total_size))
    print('\n')

print('---train_sendsize_list---')
total_size = 0
for size in train_sendsize_list:

    total_size += size
print("total train_sendsizes: {} bytes".format(total_size))
print('\n')

print('---train_receivesize_list---')
total_size = 0
for size in train_receivesize_list:

    total_size += size
print("total train_receivesizes: {} bytes".format(total_size))
print('\n')




---total_sendsize_list---
total_sendsize size: 331452604 bytes


---total_receivesize_list---
total receive sizes: 30 bytes


---client_sendsize_list(user0)---
total client_sendsizes(user0): 165726302 bytes


---client_receivesize_list(user0)---
total client_receive sizes(user0): 15 bytes


---client_sendsize_list(user1)---
total client_sendsizes(user1): 165726302 bytes


---client_receivesize_list(user1)---
total client_receive sizes(user1): 15 bytes


---train_sendsize_list---
total train_sendsizes: 331452490 bytes


---train_receivesize_list---
total train_receivesizes: 0 bytes




In [ ]:
root_path = '../../models/'

## Defining Car Dataset Class


In [ ]:
class CarDataset(object): #The code starts by initializing the CarDataset object with a list of dataframes and an image directory.

    def __init__(self, df, IMG_DIR, transforms=None):
        self.a = 0
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['img_path'].unique().tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df['img_path'] == image_id]
        image = cv2.imread(self.img_dir+image_id,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].to_numpy()
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.zeros((records.shape[0],), dtype=torch.int64)
    
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
        return image.clone().detach(), target, image_id

## Making Batch Generator

In [ ]:
batch_size = 32

### Data Augmentation

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
def train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2()
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def valid_transform():
    return A.Compose([
        ToTensorV2()
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
# Dataset path
bbox_path = './car_data/images_w_boxes.csv'
image_folder_path = './car_data/images'
data_bbox = pd.read_csv(bbox_path)
df = data_bbox.copy()

In [ ]:
image_ids = df['img_path'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]
valid_df = df[df['img_path'].isin(valid_ids)]
train_df = df[df['img_path'].isin(train_ids)]
train_df.shape,valid_df.shape

((5793, 5), (665, 5))

### `DataLoader` for batch generating

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = CarDataset(train_df, image_folder_path, train_transform())
valid_dataset = CarDataset(valid_df, image_folder_path, valid_transform())

indices = torch.randperm(len(train_dataset)).tolist()
train_data_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)


valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

### Number of total batches

In [ ]:
train_total_batch = len(train_data_loader)
print(train_total_batch)
test_batch = len(valid_data_loader)
print(test_batch)

2897
333


In [ ]:
lr = 0.001
params = [p for p in resnet_cnn.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
criterion = nn.CrossEntropyLoss()

## Inference

### Image Inference

In [ ]:
'./car_data/cars-collage-1657824102.jpg'

'./car_data/cars-collage-1657824102.jpg'

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torchvision
import torchvision.transforms as T
from collections import defaultdict, deque
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import ast
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision.transforms as transforms
import cv2
import os,sys,matplotlib,re
from PIL import Image
from skimage import exposure
import matplotlib.pyplot as plt
import matplotlib.image as immg
from torchvision.io import read_image
import matplotlib
from torchvision.utils import draw_bounding_boxes
import math

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [ ]:
# Load
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
model.load_state_dict(torch.load("./FL_fasterrcnn_car_detect.pth", map_location=torch.device(device)))
model.eval()
# model.load_state_dict(torch.load("./FL_fasterrcnn_car_detect.pth", map_location=torch.device('cpu')))

In [ ]:
# Function to Detect Cars in an Image
def detect(img_path):

    image = cv2.imread(img_path,cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
    image /= 255.0
    image = np.transpose(image, (2, 0, 1))
    image_tensor = torch.from_numpy(image)

    with torch.no_grad():
        prediction = model([image_tensor.to(device)])[0]

    keep = torchvision.ops.nms(prediction['boxes'], prediction['scores'], 0.7)
    
    final_prediction = prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]

    fig = plt.figure(figsize=(14, 10))
    img_int = torch.tensor(image_tensor * 255, dtype=torch.uint8)
    pred_img = draw_bounding_boxes(img_int, final_prediction['boxes'], width=4, colors=(255, 0, 0)).permute(1, 2, 0)
    plt.imshow(pred_img)

    return pred_img

In [ ]:
pred_img = detect("/content/drive/MyDrive/Federated_Learning/traffic_image.png")

In [ ]:
pred_img = detect("./car_data/cars-collage-1657824102.jpg")

### Video Inference

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torchvision
import torchvision.transforms as T
from collections import defaultdict, deque
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import ast
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision.transforms as transforms
import cv2
import os,sys,matplotlib,re
from PIL import Image
from skimage import exposure
import matplotlib.pyplot as plt
import matplotlib.image as immg
from torchvision.io import read_image
import matplotlib
from torchvision.utils import draw_bounding_boxes
import math

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
# Load
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
model.load_state_dict(torch.load("/content/drive/MyDrive/Federated_Learning/fasterrcnn_car_detect.pth", map_location=torch.device(device)))
model.eval()

In [ ]:
# Function to Detect Cars in one frame of the video
def detect_frame(frame):
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.float32)
    image /= 255.0
    image = np.transpose(image, (2, 0, 1))
    image_tensor = torch.from_numpy(image)

    with torch.no_grad():
        model.to(device)
        prediction = model([image_tensor.to(device)])[0]

    keep = torchvision.ops.nms(prediction['boxes'], prediction['scores'], 0.7)

    final_prediction = prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]

    # fig = plt.figure(figsize=(14, 10))
    img_int = torch.tensor(image_tensor * 255, dtype=torch.uint8)
    pred_img = draw_bounding_boxes(img_int, final_prediction['boxes'], width=4, colors=(255, 0, 0)).permute(1, 2, 0)
    # plt.imshow(pred_img)

    return pred_img

In [ ]:
# Function to check number of frames in a video
def check_frames(video_path):

    cap = cv2.VideoCapture(video_path)
    frame_no = 0
    while True:
        ret, frame = cap.read()

        if ret == True:
            frame_no += 1
        else:
            break

    print("Total number of frames in video: ",frame_no)

In [ ]:
check_frames("/content/drive/MyDrive/Federated_Learning/Traffic_5_sec.mp4")

In [ ]:
# Function to detect cars in a video
def detect_video(video_path, save_path="result.avi"):
    cap = cv2.VideoCapture(video_path)

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    size = (frame_width, frame_height)

    result = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

    if (cap.isOpened() == False): 
        print("Error reading video file")
    
    frame_no = 0
    while True:
        ret, frame = cap.read()
        if ret == True:
            output_frame = detect_frame(frame)
            numpy_frame = np.array(output_frame)
            result.write(numpy_frame)
        else:
            break
        frame_no += 1

    cap.release()
    result.release()

    print("Output Video is saved Successfully!!!")

In [ ]:
detect_video("/content/drive/MyDrive/Federated_Learning/Traffic_2_sec.mp4")